In [1]:
import requests
import datetime

from langchain.tools import StructuredTool
from langchain.pydantic_v1 import BaseModel, Field


class GetTemperaturaAtualArgs(BaseModel):
    latitude: float = Field(..., description="Latitude do local a ser consultado")
    longitude: float = Field(..., description="Longitude do local a ser consultado")

def get_temperatura_atual(latitude: float, longitude: float):
    URL = "https://api.open-meteo.com/v1/forecast"

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m",
        "forecast_days": 1
    }

    response = requests.get(URL, params=params)

    if response.status_code == 200:
        resultado = response.json()
        
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado["hourly"]["time"]]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))
        
        temp_atual = resultado["hourly"]["temperature_2m"][index_mais_prox]
        
        return temp_atual
    else:
        raise Exception(f"Erro na requisição: {response.status_code} - {response.text}")
    
    
tool_get_temperatura_atual = StructuredTool.from_function(
    func=get_temperatura_atual,
    name="tool_get_temperatura_atual",
    description="Retorna a temperatura atual em um local",
    args_schema=GetTemperaturaAtualArgs,
    return_direct=True,
)

#tool_get_temperatura_atual.run({"latitude": 52.52, "longitude": 13.41})

from typing import List
import wikipedia
wikipedia.set_lang("pt")

query = "Isaac Asimov"

class RetornaResumosWikipediaArgs(BaseModel):
    query: str = Field(..., description="Termo a ser buscado")

def busca_wikipedia(query:str) -> List:
    # titulos das paginas sobre a query
    titulos_paginas = wikipedia.search(query)

    # resumos das paginas
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(titulo, auto_suggest=True)
            resumos.append(f"Titulo da pagina: {titulo}\nResumo: {wiki_page.summary}")
        except:
            pass
    if not resumos:
        return 'Busca nao teve retorno'
    else:
        return '\n\n'.join(resumos)

tool_get_resumos_wikipedia = StructuredTool.from_function(
    func=busca_wikipedia,
    name="tool_busca_wikipedia",
    description="Retorna resumos da Wikipedia",
    args_schema=RetornaResumosWikipediaArgs,
    return_direct=True
)

# tool_get_resumos_wikipedia.invoke({"query": "futebol"})


In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Issac'),
    ('user', '{input}')
])

chat = ChatOpenAI()

tools = [tool_get_resumos_wikipedia, tool_get_temperatura_atual]
tools_json = [convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name: tool for tool in tools}

chain = prompt | chat.bind(functions=tools_json)

In [3]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

In [4]:
chain.invoke({'input': 'Olá'})

AgentFinish(return_values={'output': 'Olá! Como posso ajudar você hoje?'}, log='Olá! Como posso ajudar você hoje?')

In [5]:
chain.invoke({'input': 'Quem foi Isaac Asimov?'})

AgentActionMessageLog(tool='tool_busca_wikipedia', tool_input={'query': 'Isaac Asimov'}, log="\nInvoking: `tool_busca_wikipedia` with `{'query': 'Isaac Asimov'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Isaac Asimov"}', 'name': 'tool_busca_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 155, 'total_tokens': 176, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-3599b227-4a52-4f28-8dd6-199277a6befb-0')])

Rodando as ferramentas

In [10]:
from langchain.agents.agent import AgentFinish

def roteamento(resultado):
    if isinstance(resultado, AgentFinish):
        return resultado.return_values['output']
    else:
        return tool_run[resultado.tool].run(resultado.tool_input)

In [11]:
chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser() | roteamento

In [12]:
chain.invoke({'input': 'Olá'})

'Olá! Como posso ajudar você hoje?'

In [13]:
chain.invoke({'input': 'Quem foi Isaac Asimov?'})

'Titulo da pagina: Isaac Asimov\nResumo: Isaac Asimov (em russo: Исаак Юдович Озимов; romaniz.: Isaak Yudavich Azimov; Petrovichi, Rússia Soviética, atual Rússia, 2 de janeiro de 1920 — Brooklyn, 6 de abril de 1992) foi um escritor e bioquímico russo-americano, autor de obras de ficção científica e divulgação científica.\nAsimov é considerado um dos mestres da ficção científica e, junto com Robert A. Heinlein e Arthur C. Clarke, foi considerado um dos "três grandes" dessa área da literatura. A obra mais famosa de Asimov é a Série da Fundação, também conhecida como Trilogia da Fundação, que faz parte da série do Império Galáctico e que logo combinou com a Série Robôs. Também escreveu obras de mistério e fantasia, assim como uma grande quantidade de não-ficção. No total, escreveu ou editou mais de 500 volumes, aproximadamente 90 000 cartas ou postais, e tem obras em cada categoria importante do sistema de classificação bibliográfica de Dewey, exceto em filosofia.\nA maioria de seus livro

In [14]:
chain.invoke({'input': 'Qual a temperatura atual em São Paulo?'})

21.1